In [ ]:
import numpy as np
from scipy.stats import ortho_group
import sklearn.linear_model as lin
import time
from mpc import *
import matplotlib.pyplot as plt

In [ ]:
def subspace_basis(t, d):
    U = ortho_group.rvs(d)
    return U[:t]

# B is t x d orthogonal basis vectors, project it out of vector v
def subspace_project(B, v):
    for i in xrange(len(B)):
        v = v - np.dot(v, B[i]) * B[i]
    return v

# scale covariance down
def scaled_normal(d):
    return np.random.normal(scale = 1.0 / np.sqrt(d), size = d)

def top_vector_samples(U, n3, t):
    d = len(U)
    A1 = np.random.normal(scale = 1.0, size = (n3, d))
    diagonal = np.zeros(d)
    diagonal[:(t/2)] = np.sqrt(d) / np.sqrt(t)
    diagonal[(t/2):t] = 1.0
    basis_aligned = np.dot(U.T, np.dot(np.diag(diagonal), U))
    return np.dot(A1, basis_aligned)

# random n1 rows outside small subspace
# basis of subspace, blown up to be quite large
# other n2 random rows that are also large
def worse_instance(d, t, n1, n2, n3):
    U = ortho_group.rvs(d)
    B = U[:t]
    A1 = np.sqrt(2 * d) * top_vector_samples(U, n1, t)
    A2 = np.asarray([subspace_project(B, scaled_normal(d)) for i in xrange(n2)]) # should be unit norm?
    A3 = np.asarray([subspace_project(B, scaled_normal(d)) for i in xrange(n3)]) * d # should be norm about d
    A = np.vstack((A1, A2, A3))
    # np.random.shuffle(A)
    return A

In [ ]:
def row_norm_noise(A, xtrue, stdev):
    (n, d) = np.shape(A)
    row_norms = np.asarray([np.linalg.norm(A[i]) for i in xrange(n)])
    noise = np.dot(np.diag(row_norms), np.random.normal(scale = stdev, size = n))
    return np.dot(A, xtrue) + noise

# MPC parameters
degree = 10 # degree of rational approximation
jldirs = 7 # number of JL directions used
niters = 20 # number of iterations of solver
alpha = 1.0 # step size of solver
coeffs = np.asarray(ratapprox(degree))
reg = lin.Ridge(alpha=0.0, solver='lsqr', tol=1e-5)

# Naive and MPC solver, l2 squared error, d = 100

In [ ]:
(t, d, n1, n3) = (4, 100, 16, 50)
squared_errors = np.zeros((6, 15))
for i in xrange(6): # different values of n1
    print i
    n2 = 400 * (i + 1)
    A = worse_instance(d, t, n1, n2, n3)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(A, b).coef_
        squared_errors[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
(t, d, n1, n3) = (4, 100, 16, 50)
squared_errors_mpc = np.zeros((6, 15))
for i in xrange(6): # different values of n1
    print i
    n1 = 400 * (i + 1)
    A = worse_instance(d, t, n1, n2, n3)
    (w, total_mv) = mpc_ideal_niters(A, 500, alpha, niters, jldirs, coeffs)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
        squared_errors_mpc[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
means_squared_errors = np.asarray([np.mean(squared_errors[i]) for i in xrange(6)])
means_squared_errors_mpc = np.asarray([np.mean(squared_errors_mpc[i]) for i in xrange(6)])
stdevs_squared_errors = np.asarray([np.std(squared_errors[i]) for i in xrange(6)])
stdevs_squared_errors_mpc = np.asarray([np.std(squared_errors_mpc[i]) for i in xrange(6)])

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(400, 2800, 400), y=means_squared_errors, yerr=stdevs_squared_errors)
plt.errorbar(x=np.arange(400, 2800, 400), y=means_squared_errors_mpc, yerr=stdevs_squared_errors_mpc)
plt.xticks(np.arange(400, 2800, 400))
plt.title("Mean squared errors, d = 100")
plt.xlabel("Clean data point count")
plt.ylabel("Mean squared error")
plt.show()

# Naive and MPC solver, l2 squared error, d = 300

In [ ]:
(t, d, n1, n3) = (4, 200, 16, 100)
squared_errors_2 = np.zeros((6, 15))
for i in xrange(6): # different values of n1
    print i
    n2 = 800 * (i + 1)
    A = worse_instance(d, t, n1, n2, n3)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(A, b).coef_
        squared_errors_2[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
(t, d, n1, n3) = (4, 200, 16, 100)
squared_errors_mpc_2 = np.zeros((6, 15))
for i in xrange(6): # different values of n1
    print i
    n1 = 800 * (i + 1)
    A = worse_instance(d, t, n1, n2, n3)
    (w, total_mv) = mpc_ideal_niters(A, 1000, alpha, niters, jldirs, coeffs)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
        squared_errors_mpc_2[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
means_squared_errors_2 = np.asarray([np.mean(squared_errors_2[i]) for i in xrange(6)])
means_squared_errors_mpc_2 = np.asarray([np.mean(squared_errors_mpc_2[i]) for i in xrange(6)])
stdevs_squared_errors_2 = np.asarray([np.std(squared_errors_2[i]) for i in xrange(6)])
stdevs_squared_errors_mpc_2 = np.asarray([np.std(squared_errors_mpc_2[i]) for i in xrange(6)])

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(800, 5600, 800), y=means_squared_errors_2, yerr=stdevs_squared_errors_2)
plt.errorbar(x=np.arange(800, 5600, 800), y=means_squared_errors_mpc_2, yerr=stdevs_squared_errors_mpc_2)
plt.xticks(np.arange(800, 5600, 800))
plt.title("Mean squared errors, d = 200")
plt.xlabel("Clean data point count")
plt.ylabel("Mean squared error")
plt.show()

# Naive and MPC solver, l2 squared error, d = 300

In [ ]:
(t, d, n1, n3) = (4, 300, 16, 150)
squared_errors_3 = np.zeros((6, 15))
for i in xrange(6): # different values of n1
    print i
    n2 = 1200 * (i + 1)
    A = worse_instance(d, t, n1, n2, n3)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(A, b).coef_
        squared_errors_3[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
(t, d, n1, n3) = (4, 300, 16, 150)
squared_errors_mpc_3 = np.zeros((6, 15))
for i in xrange(6): # different values of n1
    print i
    n1 = 1200 * (i + 1)
    A = worse_instance(d, t, n1, n2, n3)
    (w, total_mv) = mpc_ideal_niters(A, 1500, alpha, niters, jldirs, coeffs)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
        squared_errors_mpc_3[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
means_squared_errors_3 = np.asarray([np.mean(squared_errors_3[i]) for i in xrange(6)])
means_squared_errors_mpc_3 = np.asarray([np.mean(squared_errors_mpc_3[i]) for i in xrange(6)])
stdevs_squared_errors_3 = np.asarray([np.std(squared_errors_3[i]) for i in xrange(6)])
stdevs_squared_errors_mpc_3 = np.asarray([np.std(squared_errors_mpc_3[i]) for i in xrange(6)])

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(1200, 8400, 1200), y=means_squared_errors, yerr=stdevs_squared_errors)
plt.errorbar(x=np.arange(1200, 8400, 1200), y=means_squared_errors_mpc, yerr=stdevs_squared_errors_mpc)
plt.xticks(np.arange(1200, 8400, 1200))
plt.title("Mean squared errors, d = 300")
plt.xlabel("Clean data point count")
plt.ylabel("Mean squared error")
plt.show()